***GENERATED CODE FOR class PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import pandas as pd
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        hdfs = HDFileSystem(host=hdfs_server, port=hdfs_port)
        with hdfs.open(eval(config)['url']) as f:
            dfPd = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PredictiveChurn_Train[State]', 'transformation_label': 'String Indexer'}], 'feature': 'PredictiveChurn_Train[State]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'AK', 'max': 'WY', 'missing': '0', 'distinct': '51'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PredictiveChurn_Train[Sta...'}, {'feature_label': 'PredictiveChurn_Train[State]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'PredictiveChurn_Train[State]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PredictiveChurn_Train[Phone]', 'transformation_label': 'String Indexer'}], 'feature': 'PredictiveChurn_Train[Phone]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '328-2236', 'max': '422-9964', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PredictiveChurn_Train[Pho...'}, {'feature_label': 'PredictiveChurn_Train[Phone]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'PredictiveChurn_Train[Phone]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PredictiveChurn_Train[cluster_labels]', 'transformation_label': 'String Indexer'}], 'feature': 'PredictiveChurn_Train[cluster_labels]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'day_callers', 'max': 'low_spenders', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PredictiveChurn_Train[clu...'}, {'feature_label': 'PredictiveChurn_Train[cluster_labels]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'PredictiveChurn_Train[cluster_labels]')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run classHooks.ipynb
try:
	#sourcePreExecutionHook()

	classhdfs = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/extra/tempFile1698992559.csv', 'filename': 'tempFile1698992559.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(classhdfs)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run classHooks.ipynb
try:
	#transformationPreExecutionHook()

	classautofe = TransformationMain.run(classhdfs,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "PredictiveChurn_Train[State]", "transformation_label": "String Indexer"}], "feature": "PredictiveChurn_Train[State]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AK", "max": "WY", "missing": "0", "distinct": "51"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PredictiveChurn_Train[Sta..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredictiveChurn_Train[Account_Length]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "101.12", "stddev": "40.07", "min": "1", "max": "225", "missing": "0"}, "updatedLabel": "PredictiveChurn_Train[Acc..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredictiveChurn_Train[Area_Code]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "413.61", "stddev": "2.79", "min": "408", "max": "415", "missing": "0"}, "updatedLabel": "PredictiveChurn_Train[Are..."}, {"transformationsData": [{"feature_label": "PredictiveChurn_Train[Phone]", "transformation_label": "String Indexer"}], "feature": "PredictiveChurn_Train[Phone]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "328-2236", "max": "422-9964", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PredictiveChurn_Train[Pho..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredictiveChurn_Train[Intl_Plan]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.09", "stddev": "0.29", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "PredictiveChurn_Train[Int..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredictiveChurn_Train[VMail_Plan]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "PredictiveChurn_Train[VMa..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredictiveChurn_Train[VMail_Message]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "PredictiveChurn_Train[VMa..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredictiveChurn_Train[Day_Mins]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "181.84", "stddev": "53.98", "min": "12.5", "max": "350.8", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PredictiveChurn_Train[Day..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredictiveChurn_Train[Day_Calls]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "101.47", "stddev": "21.65", "min": "44", "max": "158", "missing": "0"}, "updatedLabel": "PredictiveChurn_Train[Day..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredictiveChurn_Train[Day_Charge]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "30.91", "stddev": "9.18", "min": "2.13", "max": "59.64", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PredictiveChurn_Train[Day..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredictiveChurn_Train[Eve_Mins]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "204.71", "stddev": "53.14", "min": "0.0", "max": "354.2", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PredictiveChurn_Train[Eve..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredictiveChurn_Train[Eve_Calls]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "101.95", "stddev": "20.66", "min": "0", "max": "168", "missing": "0"}, "updatedLabel": "PredictiveChurn_Train[Eve..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredictiveChurn_Train[Eve_Charge]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "17.4", "stddev": "4.52", "min": "0.0", "max": "30.11", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PredictiveChurn_Train[Eve..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredictiveChurn_Train[Night_Mins]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "200.41", "stddev": "51.93", "min": "43.7", "max": "364.3", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PredictiveChurn_Train[Nig..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredictiveChurn_Train[Night_Calls]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "99.99", "stddev": "19.47", "min": "46", "max": "154", "missing": "0"}, "updatedLabel": "PredictiveChurn_Train[Nig..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredictiveChurn_Train[Night_Charge]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "9.02", "stddev": "2.34", "min": "1.97", "max": "16.39", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PredictiveChurn_Train[Nig..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredictiveChurn_Train[Intl_Mins]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "10.18", "stddev": "2.79", "min": "0.0", "max": "18.3", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PredictiveChurn_Train[Int..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredictiveChurn_Train[total_Mins]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "597.13", "stddev": "88.6", "min": "348.9", "max": "885.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PredictiveChurn_Train[tot..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredictiveChurn_Train[Intl_Calls]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "4.66", "stddev": "2.7", "min": "0", "max": "18", "missing": "0"}, "updatedLabel": "PredictiveChurn_Train[Int..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredictiveChurn_Train[Intl_Charge]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.75", "stddev": "0.75", "min": "0.0", "max": "4.94", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PredictiveChurn_Train[Int..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredictiveChurn_Train[Total_Charge]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "60.08", "stddev": "10.29", "min": "32.92", "max": "96.15", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PredictiveChurn_Train[Tot..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredictiveChurn_Train[CustServ_Calls]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "1", "missing": "0"}, "updatedLabel": "PredictiveChurn_Train[Cus..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredictiveChurn_Train[Churn]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.13", "stddev": "0.34", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "PredictiveChurn_Train[Chu..."}, {"transformationsData": [{"feature_label": "PredictiveChurn_Train[cluster_labels]", "transformation_label": "String Indexer"}], "feature": "PredictiveChurn_Train[cluster_labels]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "day_callers", "max": "low_spenders", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PredictiveChurn_Train[clu..."}]}))

	#transformationPostExecutionHook(classautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run classHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(classautofe, ["PredictiveChurn_Train[Account_Length]", "PredictiveChurn_Train[Area_Code]", "PredictiveChurn_Train[Intl_Plan]", "PredictiveChurn_Train[VMail_Plan]", "PredictiveChurn_Train[VMail_Message]", "PredictiveChurn_Train[Day_Mins]", "PredictiveChurn_Train[Day_Calls]", "PredictiveChurn_Train[Day_Charge]", "PredictiveChurn_Train[Eve_Mins]", "PredictiveChurn_Train[Eve_Calls]", "PredictiveChurn_Train[Eve_Charge]", "PredictiveChurn_Train[Night_Mins]", "PredictiveChurn_Train[Night_Calls]", "PredictiveChurn_Train[Night_Charge]", "PredictiveChurn_Train[Intl_Mins]", "PredictiveChurn_Train[total_Mins]", "PredictiveChurn_Train[Intl_Calls]", "PredictiveChurn_Train[Intl_Charge]", "PredictiveChurn_Train[Total_Charge]", "PredictiveChurn_Train[CustServ_Calls]", "PredictiveChurn_Train[State]_stringindexer", "PredictiveChurn_Train[Phone]_stringindexer", "PredictiveChurn_Train[cluster_labels]_stringindexer"], "PredictiveChurn_Train[Churn]")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

